# Booking Automation Script

This notebook automates the processing of booking requests by merging data from Google Sheets.

## Workflow
1. Connect to Google Sheets using service account credentials
2. Load data from Location, Inbox, and Output sheets
3. Validate required columns
4. Merge Inbox and Location data
5. Append to existing output and remove duplicates
6. Update the Output sheet

## Step 1: Import Libraries and Authenticate

Import required libraries and establish connection to Google Sheets using service account credentials.

In [ ]:
import gspread
import pandas as pd

# Authenticate with Google Sheets using service account
gc = gspread.service_account(filename="keys.json")

# Open the spreadsheet by ID
spreadsheet_id = "1JTHSO1t5IWqDOQxbd24vs_YMghd24ENxf4kbthUcBo8"
spreadsheet = gc.open_by_key(spreadsheet_id)

# Access individual worksheets
location_sheet = spreadsheet.worksheet("Location") 
inbox_sheet = spreadsheet.worksheet("Inbox")        
output_sheet = spreadsheet.worksheet("Output")    

# Load all records from each sheet
location_data = location_sheet.get_all_records()
inbox_data = inbox_sheet.get_all_records()
output_data = output_sheet.get_all_records()

# Convert to pandas DataFrames for easier manipulation
location_df = pd.DataFrame(location_data)
inbox_df = pd.DataFrame(inbox_data)
output_df = pd.DataFrame(output_data)

# Validate that required columns exist
if not {"Location", "Booking Number"}.issubset(location_df.columns):
    raise ValueError("Location sheet must contain 'Location' and 'Booking Number' columns.")
if not {"Location", "Contact"}.issubset(inbox_df.columns):
    raise ValueError("Inbox sheet must contain 'Location' and 'Contact' columns.")

# Merge inbox data with location data based on Location column
merged_df = pd.merge(inbox_df, location_df, on="Location", how="inner")

# If output sheet already has data, append and remove duplicates
if not output_df.empty:
    merged_df = pd.concat([output_df, merged_df], ignore_index=True)
    merged_df = merged_df.drop_duplicates()

# Clear the output sheet and write the updated data
output_sheet.clear()  
output_sheet.update([merged_df.columns.values.tolist()] + merged_df.values.tolist())

print("Output sheet updated successfully!")

Output sheet updated successfully!


## Summary

The script successfully:
- Loaded data from three Google Sheets (Location, Inbox, Output)
- Merged booking requests with location information
- Maintained historical records while removing duplicates
- Updated the Output sheet with consolidated results

The Output sheet now contains all contacts matched with their corresponding location booking numbers.